In [2]:
import numpy as np
import io
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras.layers import Dropout,Activation
from keras.layers import LSTM,SimpleRNN
from keras.callbacks import ModelCheckpoint
from keras.utils.data_utils import get_file
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
file = open("text file\\alice.txt",'r')
raw_text = []

for line in file:
    line = line.strip().lower()
    if len(line) == 0:
        continue
    raw_text.append(line)
file.close()
raw_text = "".join(raw_text)
#print(raw_text)
print('corpus length:', len(raw_text))

corpus length: 159013


In [4]:
chars = sorted(set(c for c in raw_text))
print("Length of character ",len(chars))
char2index = dict((c,i) for i,c in enumerate(chars))
index2char = dict((i,c) for i,c in enumerate(chars))

Length of character  59


In [5]:
SEQLEN = 10
STEP = 1

input_chars = []
output_chars = []

for i in range(0,len(raw_text)-SEQLEN,STEP):
    input_chars.append(raw_text[i:i+SEQLEN])
    output_chars.append(raw_text[i+SEQLEN])

In [8]:
x = np.zeros((len(input_chars),SEQLEN,len(chars)),dtype=np.bool)
y = np.zeros((len(input_chars),len(chars)),dtype=np.bool)
print(x.shape,y.shape)
for i,sentence in enumerate(input_chars):
    for j ,ch in enumerate(sentence):
        x[i,j,char2index[ch]] = 1
        y[i,char2index[output_chars[i]]] = 1

(159003, 10, 59) (159003, 59)


In [9]:
x.shape,y.shape

((159003, 10, 59), (159003, 59))

In [6]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1 
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(LSTM(128,return_sequences=False,input_shape=(SEQLEN,len(chars)),unroll=True))
model.add(Dense(len(chars)))
model.add(Activation("softmax"))

In [7]:
model.compile(loss="categorical_crossentropy",optimizer="rmsprop")

In [11]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)   
    print("Iteration #: %d" % (iteration)) 
    model.fit(x,y,epochs=NUM_EPOCHS_PER_ITERATION,batch_size=BATCH_SIZE)
    
    test_idx = np.random.randint(len(input_chars))
    text_chars = raw_text[test_idx]
    print("Generating from seed: %s" % (text_chars)) 
    print(text_chars, end="") 
    
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1,SEQLEN,len(chars)))
        
        for j ,ch in enumerate(text_chars):
            Xtest[0,j,char2index[ch]] = 1
      
        pred = model.predict(Xtest,verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred,end="")
    text_chars = text_chars[1:] + ypred
print()

Iteration #: 0
Epoch 1/1
159003/159003 [==============================] - 24s 148us/step - loss: 1.5161
Generating from seed:  
 aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa==================================================
Iteration #: 1
Epoch 1/1
159003/159003 [==============================] - 24s 151us/step - loss: 1.5097
Generating from seed: a
aiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii==================================================
Iteration #: 2
Epoch 1/1
159003/159003 [==============================] - 23s 147us/step - loss: 1.5052
Generating from seed: a
aiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii==================================================
Iteration #: 3
Epoch 1/1
159003/159003 [==============================] - 21s 135us/step - loss: 1.4993
Generating from seed: o
oyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy